## Simple Value Investing , using python (proof of concept)

This application refer [Vincent Tatan's blog at Medium](https://towardsdatascience.com/value-investing-dashboard-with-python-beautiful-soup-and-dash-python-43002f6a97ca)  to help user to makie profitable trading decision base on stock price history , financial summary , basic value analysis and infered reasonable share price. 

Most codes are based on Vincent's github project and do some modification in order to run in Jupyter notebook and  my prefered UI.  You can go to Vincent's blog to get complete explanation. 

### This application has not been rigorously tested and its domain rules are very simple which are hard to cover the complexity of real stock market. There may be also some bugs cause wrong information.
## <span style='color:blue'>Don't use this application to make real trading decision. </span>



------------------

### Main process :

1. Get stock ticker and name to be data source of dropdown control
2. Get stock price history from yahoo through pandas datareader when user choose a stock from dropdown control
3. Plot stock k chart , sma  , macd , rsi lines
4. Get stock financial information from financial website to do basic analysis and infer future value for trading decision.



### Parsing fincial information from MarketWatch web site
    For example, to get Google's financial information from this url : https://www.marketwatch.com/investing/stock/GOOG/financials



### Use pandas datareader through yahoo to get share price history
```
    pdr.DataReader(
            ticker.strip(),data_source='yahoo',start=dt(2020,1,1),end=end_date
    )
```


### Warning Signs List based on value investing logic ###

Given list of the companies, find out the feasibility to invest

1. Been in market minimal 10 years
1. Have the track records (EPS per year)
1. Have efficiency (ROE > 15%) — Net income / shareholder equity
1. Determine manipulation (ROA > 7%) — Net income / Total Asset
1. Have small long term debt (Long term debt <5* total income)
1. Low Debt to Equity
1. Ability to pay interest: (Interest Coverage Ratio >3) — EBIT / Interest expenses



### Decision rules for stock transaction

Decision making from each company in terms of return rate given the value investing methodology :

1. Find EPS Annual Compounded Growth Rate
2. Estimate EPS 10 years from now
3. Estimate stock price 10 years from now (Stock Price EPS * Average PE)
4. Determine target by price today based on returns(discount rate 15%/20%)
   - Discount rate indicated predicted inflation over the period of 5 years. 
5. Add margin of safety (Safety net 15%)
   - Margin calculation rate is the toleration for error. You add on additional safety net in the case of calculation error. The more the margin calculation rate, the more risk averse you are or vice versa.
6. Buy if market price is lower than the marginal price
7. Sell if market price is higher than the marginal price

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from pandas_datareader import data as pdr
from datetime import datetime as dt
import matplotlib as mpl
import matplotlib.pyplot as plt
import cufflinks as cf
import plotly.offline as plyo
import numpy as np
import numpy_financial as npf
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as dash_html
import dash.dependencies as dsah_depend
import ipywidgets as widgets
import plotly.graph_objects as go
import plotly.io as pio
from IPython.display import display
import dash_bootstrap_components as dbc
import webbrowser
import dashboard 
from dashboard import *
import stock_info_service 
from stock_info_service import *
import stock_info_service
from investment_analysis import *
import utils
import importlib
#importlib.reload(stock_info_service)
# from stock_info_service import *
#%load_ext autoreload
%reload_ext autoreload
%autoreload 2
#%matplotlib inline
cf.offline.go_offline() # with execute this command will cause QuantFig.iplot() run into error

In [6]:

app = JupyterDash(__name__,external_stylesheets=[dbc.themes.BOOTSTRAP])
app.layout = dashboard_layout()
app.title = 'Simple Value Investing'
register_callbacks(app)
# Run app and display result inline in the notebook
if __name__ == '__main__':
    app.run_server(
        debug=True,
        mode='external',
        port=8123,
        dev_tools_ui=True,
        dev_tools_silence_routes_logging=False) 
    # set mode='external' to use web browser
    # dev_tools_silence_routes_logging=False
    url = 'http://localhost:8123/'
    webbrowser.open(url)
    


127.0.0.1 - - [08/Sep/2020 01:35:38] "GET /_shutdown_c58f6538-fef8-403b-a059-c25195b0a6c8 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Sep/2020 01:35:38] "GET /_shutdown_c58f6538-fef8-403b-a059-c25195b0a6c8 HTTP/1.1" 200 -
127.0.0.1 - - [08/Sep/2020 01:35:38] "GET /_alive_c58f6538-fef8-403b-a059-c25195b0a6c8 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Sep/2020 01:35:38] "GET /_alive_c58f6538-fef8-403b-a059-c25195b0a6c8 HTTP/1.1" 200 -


* Running on http://127.0.0.1:8123/ (Press CTRL+C to quit)
INFO:werkzeug: * Running on http://127.0.0.1:8123/ (Press CTRL+C to quit)
